# Day 4: Sonar Sweep

[*Advent of Code 2021 day 4*](https://adventofcode.com/2021/day/4) and [*solution megathread*](https://www.reddit.com/r8i1lq)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2021/04/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2021%2F04%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys

sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

It's the day after the corporate "julbord", I haven't solved day 3 part two just yet - but at least Python is all functional on the work laptop from the hotel lobby. Guess we'll play some bingo now, how hard can it be?

For Part One, I'm calculating how many numbers must be drawn for each line (row or column) in each board to reach bingo - I bet that independence may bite me in Part Two, but going ahead for now regardless.

...

Wohoo! Part Two meant a tiny bit of refactoring and altering a `min` to a `max` - sometimes you're in luck!

Also, minutes after writing this solution I came across [Kjell Post](https://www.facebook.com/groups/242675022610339/permalink/1776252389252587) choosing a quite different path, [including numpy](https://github.com/kjepo/adventofcode/blob/main/2021/4/4a.py), which I found pretty interesting.

In [5]:
testdata = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7""".splitlines()

inputdata = downloaded['input'].splitlines()

1:80: E501 line too long (84 > 79 characters)


In [6]:
from collections.abc import Iterable
from math import log10, ceil

BingoData = list[str]
Numbers = list[int]
NumbersIdx = int
Line = Numbers
Board = list[Line]
BoardList = list[Board]
Score = int


def parse_bingo_data(data: BingoData) -> tuple[Numbers, BoardList]:
    numbers = list(map(int, data[0].split(',')))
    boards, board_current = [], []
    for line in data[2:]:
        if line != '':
            board_current.append(list(
                map(int, line.split())))
        else:
            boards.append(board_current)
            board_current = []
    boards.append(board_current)
    return numbers, boards


def board_to_string(board: BoardList) -> str:
    pad = '{{:{}d}}'.format(
        ceil(log10(
            max(i for row in board for i in row) + 1)))
    return '\n'.join(
        ' '.join(
            map(pad.format, row))
        for row in board)


def transpose(board: Board) -> Board:
    return [[row[i] for row in board]
            for i in range(len(board[0]))]


def drawn_until_bingo_line(line: Line,
                           drawn: Numbers) -> NumbersIdx:
    try:
        return max(drawn.index(digit)
                   for digit in line)
    except ValueError:
        return -1


def drawn_until_bingo_board(board: BoardList,
                            drawn: Numbers) -> NumbersIdx:
    drawn_until_bingo = min(
        filter(lambda x: x != -1,
               [drawn_until_bingo_line(line, drawn)
                for line in board + transpose(board)]))
    if not drawn_until_bingo:
        return -1
    else:
        return drawn_until_bingo


def evaluate_boards(boards: BoardList,
                    drawn: Numbers) -> list[tuple[Board, NumbersIdx]]:
    return [(board,
             drawn_until_bingo_board(board, drawn))
            for board in boards]


def winning_board(boards: BoardList,
                  drawn: Numbers):
    return min(evaluate_boards(boards,
                               drawn),
               key=lambda x: x[1])


def score_board(board: BoardList,
                drawn: Numbers,
                drawn_i: NumbersIdx) -> Score:
    final_number = drawn[drawn_i]
    drawn_win = drawn[:drawn_i + 1]
    board_numbers = set(i
                        for row in board
                        for i in row).difference(
                            drawn_win)
    return sum(board_numbers) * final_number


def my_part1_solution(data: BingoData) -> Score:
    drawn, boards = parse_bingo_data(data)
    board, drawn_i = winning_board(boards, drawn)
    return score_board(board,
                       drawn,
                       drawn_i)

In [7]:
assert(my_part1_solution(testdata) == 4512)

In [8]:
my_part1_solution(inputdata)

49860

In [9]:
HTML(downloaded['part1_footer'])

## Part Two

In [10]:
HTML(downloaded['part2'])

In [11]:
def losing_board(boards: BoardList,
                 drawn: Numbers):
    return max(evaluate_boards(boards,
                               drawn),
               key=lambda x: x[1])


def my_part2_solution(data: BingoData) -> Score:
    drawn, boards = parse_bingo_data(data)
    board, drawn_i = losing_board(boards, drawn)
    return score_board(board,
                       drawn,
                       drawn_i)

In [12]:
assert(my_part2_solution(testdata) == 1924)

In [13]:
my_part2_solution(inputdata)

24628

In [14]:
HTML(downloaded['part2_footer'])